## Image classification using augmentation and dropout in Keras

In [1]:
%matplotlib inline
#from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from imp import reload

In [2]:
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image
from matplotlib import pyplot as plt
from utils import plots, get_batches, plot_confusion_matrix, get_data
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [3]:
import utils; reload(utils)
from utils import *

In [6]:
#path = "data/dogscats/"
path = "data/dogscats/sample/"

In [7]:
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [8]:
model = vgg_ft_bn(2) #out_dim =2 

C:\home\dogcat\vgg16bn.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
C:\home\dogcat\vgg16bn.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
C:\home\dogcat\vgg16bn.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
C:\home\dogcat\vgg16bn.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))


In [9]:
layers = model.layers

In [10]:
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

In [11]:
last_conv_idx

30

In [12]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)

In [13]:
fc_layers = layers[last_conv_idx+1:]

In [14]:
for layer in conv_model.layers: layer.trainable = False

In [15]:
p = 0.4

In [16]:
opt = RMSprop(lr=0.00001, rho=0.7)

In [44]:
def get_fc_model(p):
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
        ])
    return model

In [46]:
fc_model = get_fc_model(p)

In [19]:
conv_model.add(fc_model)

In [24]:
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [25]:
batch_size = 8

In [26]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

In [27]:
batches = get_batches(path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [28]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [29]:
batches.class_indices

{'cats': 0, 'dogs': 1}

In [30]:
batches.samples

160

In [31]:
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
conv_model.fit_generator(batches, samples_per_epoch=batches.samples/batch_size, epochs=2, 
                        validation_data=val_batches, validation_steps = val_batches.samples/batch_size)

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=2, steps_per_epoch=2.0, validation_data=<keras.pre..., validation_steps=5.0)`
  from ipykernel import kernelapp as app


Epoch 1/2
2/2 [==============================] - 155s - loss: 7.3276 - acc: 0.3750 - val_loss: 1.9033 - val_acc: 0.7000
Epoch 2/2
2/2 [==============================] - 145s - loss: 4.0377 - acc: 0.5625 - val_loss: 1.6740 - val_acc: 0.7500


In [50]:
p = 0.5

In [51]:
conv_model.fit_generator(batches, samples_per_epoch=batches.samples/batch_size, epochs=1, 
                        validation_data=val_batches, validation_steps = val_batches.samples/batch_size)
#conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=3, 
                        #validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=1, steps_per_epoch=2.0, validation_data=<keras.pre..., validation_steps=5.0)`
  from ipykernel import kernelapp as app


Epoch 1/1
2/2 [==============================] - 165s - loss: 1.7475 - acc: 0.6250 - val_loss: 1.7074 - val_acc: 0.6250


In [64]:
val_data = get_data(path+'valid')

Found 40 images belonging to 2 classes.


In [65]:
prob_preds = conv_model.predict_proba(val_data, batch_size = batch_size)

40/40 [==============================] - 93s     
